# OUTLINE
 1 Python basics : Setup, syntax & standard libraries
        
 __2 Pymatgen 1 : build a structure & write a vasp job__
    
 3 Python + slurm + bash : launch a vasp job
    
 4 Pymatgen 2 : read a vasp job & plot DOS
    
 5 Matplotlib : make your own plots 
    
 6 Pymatgen 3 : advanced functionalities (disorder, bader, lobster ... )
    
 7 the read_write library : installation, work flow & examples


# Session 2 : Pymatgen  1 : reading a poscar and writing vasp input
## The Structure module [doc](http://pymatgen.org/pymatgen.core.structure.html) [source code](http://pymatgen.org/_modules/pymatgen/core/structure.html)
* Reading & writing POSCAR or CIF data 
* Modifying a Structure 
## the VaspInputSet module [doc](http://pymatgen.org/pymatgen.io.vasp.sets.html?highlight=vaspinputset)
* Write vasp inputs into a directory (INCAR,POSCAR,KPOINT,POTCAR)
* Modyfiying the default settings
## the Job class


## Lattice & Structure objects

A Structure object represents a crystal structure (lattice + basis). 

A Structure is essentially a list of `PeriodicSites` with the same `Lattice`.

`PeriodicSites` & `Lattice` are both core classes of pymatgen.

Let us now create a CsCl structure from scratch.

A Lattice represents a Bravais lattice. Convenience static functions are provided for the creation of common lattice types from a minimum number of arguments.

In [8]:
import pymatgen as mg
# Creates cubic Lattice with lattice parameter 4.2
lattice = mg.Lattice.cubic(4.2)
print(lattice.parameters)

(4.2, 4.2, 4.2, 90.0, 90.0, 90.0)


In [17]:
structure = mg.Structure(lattice, ["Na", "Cl"], [[0, 0, 0], [0.5, 0.5, 0.5]])
print("Unit cell vol = {}".format(structure.volume))
print("First site of the structure is {}".format(structure[0]))

Unit cell vol = 74.08800000000001
First site of the structure is [0. 0. 0.] Na


The Structure object contains many useful manipulation functions. Since Structure is essentially a list, it contains a simple pythonic API for manipulation its sites.

In [3]:
structure.make_supercell([2, 1, 1]) #Make a 2 x 1 x 1 supercell of the structure
del structure[0] #Remove the first site
structure.append("Na", [0,0,0]) #Append a Na atom.
structure[-1] = "Li" # Change the last added atom to Li.
structure[0] = "Na", [0.01, 0.5, 0] #Shift the first atom by 0.01 in fractional coordinates in the x-direction.
structure.replace_species({"Cl" : "F"}) # replace all species
print("F indices", structure.indices_from_symbol("F")) # find all F indices
print(structure)

F indices (1, 2)
Full Formula (Na1 Li1 F2)
Reduced Formula: NaLiF2
abc   :   8.400000   4.200000   4.200000
angles:  90.000000  90.000000  90.000000
Sites (4)
  #  SP       a    b    c
---  ----  ----  ---  ---
  0  Na    0.01  0.5  0
  1  F     0.25  0.5  0.5
  2  F     0.75  0.5  0.5
  3  Li    0     0    0


#### Adding custom properties 
The method `structure.add_site_property(property_name: str, values: List)` is very usefull : 
* to store site-related information (e.g. bader charge) for further analysis (bader-oxidation ewald analysis)
* for advanced modification of the input files for vasp (cf below) : 

    `"spin"` , `[ +3, 0, ... ]` => (define the magmom in the incar )

    `"selective dynamic"` ,  `[ [ True, True, False], [...] ... ]` => define the degree of freedom in POSCAR

We will se examples at the end of this tutorial




## Basic analyses

Pymatgen provides many analyses functions for Structures. Some common ones are given below.

### Determining the symmetry 

In [26]:
# quick method directly from a structure : 
print("The spacegroup is {}, with number {}".format( *structure.get_space_group_info() ))

# behind the scene we use the SpacegroupAnalyzer class which has MUCH more functionalities
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
finder = SpacegroupAnalyzer(structure)
print("The point group operation (displacement + rotation) matices are \n {}".format( finder.get_point_group_operations() ))

The spacegroup is Pm-3m, with number 221
The point group operation (displacement + rotation) matices are 
 [Rot:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
tau
[0. 0. 0.], Rot:
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.], Rot:
[[ 0. -1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.], Rot:
[[ 0.  1.  0.]
 [-1.  0.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.], Rot:
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.], Rot:
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.], Rot:
[[ 0.  1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.], Rot:
[[ 0. -1.  0.]
 [ 1.  0.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.], Rot:
[[ 1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.], Rot:
[[-1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.], Rot:
[[ 0. -1.  0.]
 [-1.  0.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.], Rot:
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
tau
[0. 0. 0.], Rot:
[[-1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.], Rot:
[[ 1.  0.  0.]
 [ 0. -1.  0.]
 [ 0

### Matching structures
useful to elimiate doubles when enumerating disordered structures (see beslow)

In [33]:
#Let's create two structures which are the same topologically, but with different elements, and one lattice is larger.
s1 = mg.Structure(lattice, ["Cs", "Cl"], [[0, 0, 0], [0.5, 0.5, 0.5]])
s2 = mg.Structure(mg.Lattice.cubic(5), ["Rb", "F"], [[0, 0, 0], [0.5, 0.5, 0.5]])

# convenient method from the Structure instance
print(s1.matches(s2, anonymous=True))

# behind the scene we use the StructureMatcher class, with other tools avaliable
from pymatgen.analysis.structure_matcher import StructureMatcher
m = StructureMatcher()
print(m.fit_anonymous(s1, s2)) #Returns a mapping which maps s1 and s2 onto each other. Strict element fitting is also available.

True
True


#### Distances across the structure 
usefull when studying the deformation 

In [72]:
structure = mg.Structure.from_file("LiCoO2.cif")

# get O-O distances
O_indices = structure.indices_from_symbol("O")
oo_dist_list = []
for i, O1 in enumerate(O_indices) : 
    for O2 in O_indices[i+1 : ]: # upper diagonal of the matrix to avoid doubles 
        oo_dist_list.append([O1,O2,structure.distance_matrix[O1,O2]])

# sort O-O distances 
oo_dist_list = sorted(oo_dist_list, key=lambda x: x[2]) #  sort function and lambda function 
for O1,O2,dist in oo_dist_list : 
    print("dist [ {} , {} ] = {}".format(O1,O2,structure.distance_matrix[O1,O2]))

# get oxygen in cobalt first shell
co_site = structure[structure.indices_from_symbol("Co")[0]] # PeriodicSite instance
structure.get_neighbors(co_site, 2.5)

dist [ 7 , 10 ] = 2.343920428378692
dist [ 6 , 9 ] = 2.3439204283786927
dist [ 8 , 11 ] = 2.343920428378693
dist [ 6 , 11 ] = 3.405663221503893
dist [ 7 , 8 ] = 3.405663221503893
dist [ 9 , 10 ] = 3.4056632215038953
dist [ 8 , 10 ] = 4.956500692132615
dist [ 9 , 11 ] = 4.956500692132616
dist [ 7 , 11 ] = 4.956500692132617
dist [ 6 , 8 ] = 4.956500692132618
dist [ 6 , 10 ] = 4.956500692132618
dist [ 7 , 9 ] = 4.956500692132618
dist [ 10 , 11 ] = 6.3721728
dist [ 6 , 7 ] = 6.372172800000001
dist [ 8 , 9 ] = 6.372172800000001


[PeriodicSite: O (-1.4070, -0.8123, -1.4966) [-0.6667, -0.3333, -0.1065],
 PeriodicSite: O (-1.4070, 0.8123, 1.4966) [-0.3333, 0.3333, 0.1065],
 PeriodicSite: O (1.4070, -0.8123, -1.4966) [0.3333, -0.3333, -0.1065],
 PeriodicSite: O (-0.0000, -1.6247, 1.4966) [-0.3333, -0.6667, 0.1065],
 PeriodicSite: O (0.0000, 1.6247, -1.4966) [0.3333, 0.6667, -0.1065],
 PeriodicSite: O (1.4070, 0.8123, 1.4966) [0.6667, 0.3333, 0.1065]]

## Input/output

In [6]:
#Convenient IO to various formats. Format is intelligently determined from file name and extension.
structure.to(filename="POSCAR")
structure.to(filename="CsCl.cif")

#Or if you just supply fmt, you simply get a string.
print(structure.to(fmt="poscar"))
print(structure.to(fmt="cif"))

Na1 Li1 F2
1.0
8.400000 0.000000 0.000000
0.000000 4.200000 0.000000
0.000000 0.000000 4.200000
Na F Li
1 2 1
direct
0.010000 0.500000 0.000000 Na
0.250000 0.500000 0.500000 F
0.750000 0.500000 0.500000 F
0.000000 0.000000 0.000000 Li

# generated using pymatgen
data_NaLiF2
_symmetry_space_group_name_H-M   'P 1'
_cell_length_a   8.40000000
_cell_length_b   4.20000000
_cell_length_c   4.20000000
_cell_angle_alpha   90.00000000
_cell_angle_beta   90.00000000
_cell_angle_gamma   90.00000000
_symmetry_Int_Tables_number   1
_chemical_formula_structural   NaLiF2
_chemical_formula_sum   'Na1 Li1 F2'
_cell_volume   148.17600000
_cell_formula_units_Z   1
loop_
 _symmetry_equiv_pos_site_id
 _symmetry_equiv_pos_as_xyz
  1  'x, y, z'
loop_
 _atom_site_type_symbol
 _atom_site_label
 _atom_site_symmetry_multiplicity
 _atom_site_fract_x
 _atom_site_fract_y
 _atom_site_fract_z
 _atom_site_occupancy
  Na  Na0  1  0.010000  0.500000  0.000000  1
  F  F1  1  0.250000  0.500000  0.500000  1.0
  F  F2  1  

In [7]:
# Reading a structure from a file.
structure = mg.Structure.from_file("POSCAR")

### Exercise 
#### 1 read the cif of LiCoO2

#### 2 replace half Co by Mn (random replacement)
* you may have to build asupercell if there is less than 6 Li in the provided cell
#### 3 remove x=1/6 Li  
* random removal : use the `random` library

#### 4 write the corresponding POSCAR

In [8]:
# space for exercise here !! 

### Solving the exercice "a la mano", 
ordering = random

In [83]:
structure = mg.Structure.from_file("LiCoO2.cif")
big_structure = structure.copy()

import random, math

while True :
    Li_indices = big_structure.indices_from_symbol("Li")
    nb_li = len(Li_indices)
    co_indices = big_structure.indices_from_symbol("Co")
    nb_co = len(co_indices)
    print(" \nNb of Li : {} // nb co {}".format(nb_li, nb_co))
    if nb_li // 6 > 0 and nb_co // 2 > 0 == 0 :
        print("enough li & Mn")
        break
    big_structure.make_supercell([2, 1, 1])    
    print("no enough li or Mn, making supercell")

print(big_structure)

nb_co_replace = int(nb_co//2) 
random_indices_to_replace = random.sample(co_indices, nb_co_replace)
print("\nindices Co to replace :",  random_indices_to_replace)
for i in random_indices_to_replace : 
    big_structure.replace(i,"Mn")

nb_li_remove = int(nb_co//6) 
random_indices_to_remove = random.sample(Li_indices, nb_li_remove)
print("\n indices Li to remove :",  random_indices_to_remove)
big_structure.remove_sites(random_indices_to_remove)

print(big_structure)


Nb of Li : 3 // nb co 3
no enough li or Mn, making supercell
 
Nb of Li : 6 // nb co 6
enough li & Mn
Full Formula (Li6 Co6 O12)
Reduced Formula: LiCoO2
abc   :   5.628000   2.814000  14.048000
angles:  90.000000  90.000000 120.000000
Sites (24)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  Li    0         0         0.5
  1  Li    0.5       0         0.5
  2  Li    0.333333  0.333333  0.833333
  3  Li    0.833333  0.333333  0.833333
  4  Li    0.166667  0.666667  0.166667
  5  Li    0.666667  0.666667  0.166667
  6  Co    0         0         0
  7  Co    0.5       0         0
  8  Co    0.333333  0.333333  0.333333
  9  Co    0.833333  0.333333  0.333333
 10  Co    0.166667  0.666667  0.666667
 11  Co    0.666667  0.666667  0.666667
 12  O     0         0         0.2268
 13  O     0.5       0         0.2268
 14  O     0         0         0.7732
 15  O     0.5       0         0.7732
 16  O     0.333333  0.333333  0.560133
 17  O     0.833333  0.33

### Solving the exercice using pymatgen builtin ordering functions  

In [7]:
structure = mg.Structure.from_file("LiCoO2.cif")
disordered_structure = structure.copy()

disordered_structure.replace_species({"Co" : {"Mn":0.5, "Co" : 0.5}}) # metal disorder
disordered_structure.replace_species({"Li" : {"Li":5/6}}) # partial Li vacancy

enumeration of ALL distinct structures
* search for large supercell if necessary (max cell size or nb of sites can be setup)
* ordered by size (smallest first)

In [8]:
from pymatgen.transformations.advanced_transformations import EnumerateStructureTransformation

trans = EnumerateStructureTransformation(max_cell_size=3)
ss = trans.apply_transformation(disordered_structure,return_ranked_list=1000)

ss[0]
# supercell 211

{'num_sites': 23, 'structure': Structure Summary
 Lattice
     abc : 2.814 2.814000000000001 28.096
  angles : 90.0 90.0 59.999999999999986
  volume : 192.67408806120096
       A : 2.814 0.0 1.723078046400326e-16
       B : 1.407000000000001 2.436995486249411 3.446156092800652e-16
       C : 0.0 0.0 28.096
 PeriodicSite: Li (0.0000, 0.0000, 7.0240) [0.0000, 0.0000, 0.2500]
 PeriodicSite: Li (0.0000, 0.0000, 21.0720) [0.0000, 0.0000, 0.7500]
 PeriodicSite: Li (1.4070, 0.8123, 11.7067) [0.3333, 0.3333, 0.4167]
 PeriodicSite: Li (1.4070, 0.8123, 25.7547) [0.3333, 0.3333, 0.9167]
 PeriodicSite: Li (2.8140, 1.6247, 2.3413) [0.6667, 0.6667, 0.0833]
 PeriodicSite: Mn (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
 PeriodicSite: Mn (0.0000, 0.0000, 14.0480) [0.0000, 0.0000, 0.5000]
 PeriodicSite: Mn (1.4070, 0.8123, 18.7307) [0.3333, 0.3333, 0.6667]
 PeriodicSite: Co (1.4070, 0.8123, 4.6827) [0.3333, 0.3333, 0.1667]
 PeriodicSite: Co (2.8140, 1.6247, 9.3653) [0.6667, 0.6667, 0.3333]
 Period

ordering = lowest ewald sum 
* fixed structure size 
* provide (guess) atomic oxidation states

In [44]:
from pymatgen.transformations.standard_transformations import OrderDisorderedStructureTransformation
big_structure = disordered_structure.copy()
big_structure.make_supercell([2, 1, 1]) 
big_structure.add_oxidation_state_by_guess()
# print(big_structure)
trans = OrderDisorderedStructureTransformation()

ss = trans.apply_transformation(big_structure, return_ranked_list=100)

 
print(ss[0])

{'energy': -552.4665447802587, 'energy_above_minimum': 0.0, 'structure': Structure Summary
Lattice
    abc : 5.628 2.814 14.048
 angles : 90.0 90.0 119.99999999999999
 volume : 192.67408806120096
      A : 5.628 0.0 3.446156092800652e-16
      B : -1.4069999999999991 2.436995486249411 1.723078046400326e-16
      C : 0.0 0.0 14.048
PeriodicSite: Li+ (0.0000, 0.0000, 7.0240) [0.0000, 0.0000, 0.5000]
PeriodicSite: Li+ (1.4070, 0.8123, 11.7067) [0.3333, 0.3333, 0.8333]
PeriodicSite: Li+ (4.2210, 0.8123, 11.7067) [0.8333, 0.3333, 0.8333]
PeriodicSite: Li+ (0.0000, 1.6247, 2.3413) [0.1667, 0.6667, 0.1667]
PeriodicSite: Li+ (2.8140, 1.6247, 2.3413) [0.6667, 0.6667, 0.1667]
PeriodicSite: Mn3.66666667+ (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Mn3.66666667+ (1.4070, 0.8123, 4.6827) [0.3333, 0.3333, 0.3333]
PeriodicSite: Mn3.66666667+ (2.8140, 1.6247, 9.3653) [0.6667, 0.6667, 0.6667]
PeriodicSite: Co2.66666667+ (2.8140, 0.0000, 0.0000) [0.5000, 0.0000, 0.0000]
PeriodicSite:

## VASP INPUTS 

### THE DEFAULT PYMATGEN PRESETS 

The `pymatgen.io.vasp.sets` module provides a means to obtain a complete set of VASP input files for performing calculations. 

Several presets based on the parameters used in the Materials Project are provided. They can all be changed easily. 



#### MPRelaxSet (material Project Relaxation set)

In [81]:
from pymatgen.io.vasp.sets import MPRelaxSet
structure = mg.Structure.from_file("LiCoO2.cif")
relax = MPRelaxSet(structure)

# option for nice formatting of oxides formulas 
pretty_compo = structure.composition.get_reduced_formula_and_factor()[0]
print(pretty_compo)

#Writes a complete set of input files for structure to the directory given
relax.write_input("{}_Vasp_input_default".format(pretty_compo)) 

LiCoO2


#### MPStaticSet
Generate VASP Input for Static Run

In [0]:
from pymatgen.io.vasp.sets import MPStaticSet

custom_settings = {"NELM": 60} # user custom incar settings
static = MPStaticSet.from_prev_calc("Si-relax/", standardize=True,
                                    user_incar_settings=custom_settings)
static.write_input("Si-static")

### MPNonSCFSet
Generate VASP Input for Density of State(DOS) and Band Structure(BS) calculation

In [0]:
from pymatgen.io.vasp.sets import MPNonSCFSet

# generate uniform k-points for DOS calc.
custom_settings = {"LAECHG": "False", "LVHAR": "False"} # user custom incar settings
dos = MPNonSCFSet.from_prev_calc("Si-static/", mode="uniform",reciprocal_density=200,
                                 user_incar_settings=custom_settings)
dos.write_input("Si-dos")

# generate k-points along high symmetry line for band structure calc.
band = MPNonSCFSet.from_prev_calc("Si-static/", mode="line", standardize=True,
                                 user_incar_settings=custom_settings)
band.write_input("Si-band")

### PERSONALIZATION OF YOUR VASP INPUT SET
We can specify each of the parameters (INCAR, KPOINTS, POTCAR)

In [86]:
incar_settings = {"NELMIN": 5} # user custom incar settings

kpoint_settings = {"reciprocal_density" : 100}

# NB : may require to setup custor POTCAR repository
functional = "PBE_54" # “PBE”, “PBE_52”, “PBE_54”, “LDA”, “LDA_52”, “LDA_54”, “PW91”, “LDA_US”, “PW91_US”

relax = MPRelaxSet(structure, 
        user_incar_settings=incar_settings, 
        user_kpoints_settings = kpoint_settings, 
        potcar_functional=functional )

relax.write_input("{}_Vasp_input_modified".format(pretty_compo)) 

#### Advanced parametrization, based on atom-wise properties
Individual Magmoms

In [80]:
structure = mg.Structure.from_file("LiCoO2.cif")
structure.add_spin_by_element({"Li":0,"Co":+3,"O":0} )
print(structure)
relax = MPRelaxSet(structure)
relax.write_input("{}_Vasp_input_magmoms".format(pretty_compo)) 

# => check magmom tag in the INCAR

Full Formula (Li3 Co3 O6)
Reduced Formula: LiCoO2
abc   :   2.814000   2.814000  14.048000
angles:  90.000000  90.000000 120.000000
Sites (12)
  #  SP                a         b         c
---  ---------  --------  --------  --------
  0  Li,spin=0  0         0         0.5
  1  Li,spin=0  0.666667  0.333333  0.833333
  2  Li,spin=0  0.333333  0.666667  0.166667
  3  Co,spin=3  0         0         0
  4  Co,spin=3  0.666667  0.333333  0.333333
  5  Co,spin=3  0.333333  0.666667  0.666667
  6  O,spin=0   0         0         0.2268
  7  O,spin=0   0         0         0.7732
  8  O,spin=0   0.666667  0.333333  0.560133
  9  O,spin=0   0.666667  0.333333  0.106533
 10  O,spin=0   0.333333  0.666667  0.893467
 11  O,spin=0   0.333333  0.666667  0.439867


Selective dynamic 

In [77]:
# we decide to move only the Li in the X-Y plane : 
structure = mg.Structure.from_file("LiCoO2.cif")
select_dyn_array = np.zeros((structure.num_sites, 3), dtype = bool)
for x in structure.indices_from_symbol("Li") :
    select_dyn_array[x,:2]=True
# print(select_dyn_array)
structure.add_site_property("selective_dynamics", select_dyn_array)
print(structure)
relax = MPRelaxSet(structure)
relax.write_input("{}_Vasp_input_selective_dynamic".format(pretty_compo)) 

# => check selective dyn in the POSCAR 

Full Formula (Li3 Co3 O6)
Reduced Formula: LiCoO2
abc   :   2.814000   2.814000  14.048000
angles:  90.000000  90.000000 120.000000
Sites (12)
  #  SP           a         b         c  selective_dynamics
---  ----  --------  --------  --------  --------------------
  0  Li    0         0         0.5       [ True  True False]
  1  Li    0.666667  0.333333  0.833333  [ True  True False]
  2  Li    0.333333  0.666667  0.166667  [ True  True False]
  3  Co    0         0         0         [False False False]
  4  Co    0.666667  0.333333  0.333333  [False False False]
  5  Co    0.333333  0.666667  0.666667  [False False False]
  6  O     0         0         0.2268    [False False False]
  7  O     0         0         0.7732    [False False False]
  8  O     0.666667  0.333333  0.560133  [False False False]
  9  O     0.666667  0.333333  0.106533  [False False False]
 10  O     0.333333  0.666667  0.893467  [False False False]
 11  O     0.333333  0.666667  0.439867  [False False False]


## the Job class
The built-in InputSet class is quite limited because once it is created we cannot change anything.
 Furthermore the incar defaults might differ from what we want. 

However, we can create a child class, the Job class, which inherits from the InputSet class.
Below is an extract of the Job class definition in the module write_job. 

its has more possibilities such as : 
* personalized default incar
* modify the structure, incar, kpoints, ... until we write the files down in the folder
* duplicate a Job
* create a job from a converged run (Rundict class, see session 4)
* pretty formatting for the name of the folder
* possible modification of the structure 

and we can still add more ! 

have a look at the write_job module if you are interested

In [0]:
class Job(MPRelaxSet):
    """Describe a VASP run BEFORE computation

        Holds and modifies VASP inputs (Structure, INCAR, Kpoint,...)
        before writing the files in the folder

        lazy use of MPRelaxSet .__init__ & .write to create files"""

    @property
    def structure(self):
        """Bypass (overload) structure access restricted in pmg.MPRelaxSet"""
        return self._structure

    @structure.setter
    def structure(self, value):
        """"Bypass (overload) structure modification restricted in pmg.MPRelaxSet"""
        self._structure = value

    def __init__(self, structure, entry_id,
                 user_param=None, user_incar=None,
                 job_folder="", **kwargs):
        """Generates a runDict from a Computed entry if it is none None"""

        self.explicit_jobpath = False
        self.entry_id = entry_id
        self.job_folder = job_folder

        self.user_kpoint = {'reciprocal_density': 100}
        self.user_param = user_param
        self.user_incar = user_incar if user_incar is not None \
            else default_incar()

        self.kwargs = kwargs
        for (key, value) in kwargs.items():
            setattr(self, key, value)

        self.standardize = False
        self.structure = structure

        self.nb_cell = 1
        self.x_na = 0
        self.formula = ""
        self.old_folder = None
        self.job_name = self.set_job_name()